In [ ]:
import sys

sys.path.append("../")

from peakweather.dataset import PeakWeatherDataset
from src.data import plot_stations_on_map, plot_timeseries

### Caricare i dati
La classe `PeakWeatherDataset` permette di scaricare e gestire i dati meteorologici del progetto PeakWeather.
In questo esempio, scarichiamo i dati orari delle stazioni meteorologiche, calcolando la media della temperatura.

In [ ]:
ds = PeakWeatherDataset(
    root=None,
    compute_uv=False,
    station_type="meteo_station",
    freq="h",
    aggregation_methods={"temperature": "mean"},
)

### Esplorare il dataset
Visualizziamo alcune informazioni sul dataset, come i parametri disponibili e le stazioni presenti.

In [ ]:
print(ds)
print(f"Parameters")
ds.show_parameters_description()

In [ ]:
print(f"Stations")
print(ds.stations.values)

#### Domanda
Quante stazioni meteorologiche sono presenti nel dataset?

### Come è fatta una serie temporale?
Visualizziamo in forma numerica le series temporali

In [ ]:
ds.get_observations(stations="ABO", parameters="temperature").head()

### Visualizzare le stazioni su una mappa
Utilizziamo la funzione `plot_stations_on_map` per visualizzare le stazioni meteorologiche su una mappa.
Possiamo specificare un sottoinsieme di stazioni fornendo una lista di ID delle stazioni.

In [ ]:
plot_stations_on_map(dataset=ds, station_ids=None)

#### Esercizio
Scegli un sottoinsieme di stazioni meteorologiche e visualizzale sulla mappa.

Puoi creare una lista di ID delle stazioni, ad esempio: `station_ids = ["ABO", "AEG", "AIG"]`, e passarla alla funzione `plot_stations_on_map` al posto di `None`.

In [ ]:
# Inserisci qui il tuo codice
# station_ids = ???
# plot_stations_on_map(dataset=ds, station_ids=???)

### Visualizzare serie temporali
Utilizziamo la funzione `plot_timeseries` per visualizzare le serie temporali di una determinata variabile meteorologica per un insieme di stazioni e un intervallo di date specificato.

In [ ]:
plot_timeseries(
    dataset=ds,
    station_ids=["ABO", "AEG", "AIG"],
    parameter="temperature",
    start_date="2020-01-01",
    end_date="2020-01-31",
)

### Noti qualche problema?
Per alcune stazioni, potrebbero esserci dei dati mancanti in alcuni periodi o per alcune variabili.

#### Esercizio
- Scegli la tua stazione meteorologica preferita e visualizza la serie temporale della temperatura per il mese di luglio 2020. (Hint: il formato della data è "YYYY-MM-DD".)
- Prova a cambiare l'intervallo di date per vedere come variano i dati. (Hint: Le date disponibili variano tra Gennaio 2017 e Marzo 2025.)
- Prova a visualizzare un'altra variabile meteorologica per la stessa stazione e periodo. (Hint: abbiamo visualizzato i nomi delle variabili in precedenza.)
- Riconosci qualche pattern stagionale o tendenza nei dati?

In [ ]:
# Inserisci qui il tuo codice
# plot_timeseries(
#     dataset=ds,
#     station_ids=[???],
#     parameter=???,
#     start_date=???,
#     end_date=???,
# )